In [7]:
!sudo apt update
!sudo apt install -y build-essential autoconf libtool pkg-config python3-dev \
    python3-pip python3-numpy git flex bison libbz2-dev

!wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | sudo apt-key add -
!sudo apt-add-repository 'deb https://apt.kitware.com/ubuntu/ bionic main'
!sudo apt-get update && apt-get --allow-unauthenticated install -y \
    cmake \
    kitware-archive-keyring

# feel free to use a more elegant solution to make /usr/bin/cmake the default one
!sudo rm $(which cmake)
!$(which cmake) --version

!pip3 install -Uv nle
!apt-get install sox ffmpeg libcairo2 libcairo2-dev
!pip install manimlib pygame opencv-python minihack

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://apt.kitware.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://apt.kitware.com/ubuntu/dists/bio

In [7]:
!unzip dqn.zip

Archive:  dqn.zip
   creating: dqn/
  inflating: dqn/model.py            
  inflating: dqn/wrappers.py         
  inflating: dqn/agent.py            
  inflating: dqn/replay_buffer.py    
   creating: dqn/__pycache__/
  inflating: dqn/__pycache__/replay_buffer.cpython-38.pyc  
  inflating: dqn/__pycache__/wrappers.cpython-38.pyc  
  inflating: dqn/__pycache__/agent.cpython-38.pyc  
  inflating: dqn/__pycache__/visualize.cpython-38.pyc  
  inflating: dqn/__pycache__/model.cpython-38.pyc  
  inflating: dqn/visualize.py        
  inflating: train_dqn.py            


In [1]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://apt.kitware.com/ubuntu bionic InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: https://apt.kitware.com/ubuntu/dists/bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.

In [2]:
import stable_baselines3

print(f"{stable_baselines3.__version__=}")

stable_baselines3.__version__='2.1.0'


In [3]:
import gymnasium as gym
import numpy as np

print(f"{gym.__version__=}")

gym.__version__='0.29.1'


In [4]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

In [ ]:
import random
import numpy as np
import gym

from dqn.agent import DQNAgent
from dqn.replay_buffer import ReplayBuffer
from dqn.wrappers import *
from dqn.visualize import make_video
from nle import nethack
from minihack import RewardManager
import torch
from stable_baselines3 import A2C
hyper_params = {
    "seed": 69,  # which seed to use
    "env": "MiniHack-LavaCross-Levitate-Potion-Inv-Full-v0",  # name of the game
    "replay-buffer-size": int(1e6),  # replay buffer size
    "learning-rate": 0.001,  # learning rate for Adam optimizer
    "discount-factor": 0.99,  # discount factor
    "num-steps": int(30000),  # total number of steps to run the environment for
    "batch-size": 256,  # number of transitions to optimize at the same time
    "learning-starts": 1000,  # number of steps before learning starts
    "learning-freq": 2,  # number of iterations between every optimization step
    "use-double-dqn": True,  # use double deep Q-learning
    "target-update-freq": 1000,  # number of iterations between every target network update
    "eps-start": 1.0,  # e-greedy start threshold
    "eps-end": 0.4,  # e-greedy end threshold
    "eps-fraction": 0.95,  # fraction of num-steps
    "print-freq": 10,
}


np.random.seed(hyper_params["seed"])
random.seed(hyper_params["seed"])

# ACTIONS = nethack.CompassDirection.all_directions() + nethack.Command.all_directions()
ACTIONS = (
    nethack.CompassDirection.N,
    nethack.CompassDirection.E,
    nethack.CompassDirection.S,
    nethack.CompassDirection.W,
    nethack.Command.QUAFF,
    nethack.Command.FIRE,
    # nethack.Command.PICKUP,
    # nethack.Command.APPLY
    )

env = gym.make(hyper_params["env"],
                observation_keys = ['pixel', 'message', 'glyphs'],
                # penalty_time=-0.1,
                # penalty_step=-0.1,
                reward_lose=-3,
                reward_win=5,
                # seeds = hyper_params["seed"],
                actions = ACTIONS
                )
# env = gym.make("MiniHack-LavaCross-Levitate-Potion-Inv-Full-v0")
model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    # vec_env.render("human")
    # VecEnv resets automatically
    # if done:
    #   obs = vec_env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 186      |
|    ep_rew_mean        | 4.32     |
| time/                 |          |
|    fps                | 32       |
|    iterations         | 100      |
|    time_elapsed       | 15       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.62    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.00706 |
|    value_loss         | 6.43e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 196      |
|    ep_rew_mean        | 4.27     |
| time/                 |          |
|    fps                | 33       |
|    iteratio

In [8]:
import random
import numpy as np
import gym

from dqn.agent import DQNAgent
from dqn.replay_buffer import ReplayBuffer
from dqn.wrappers import *
from dqn.visualize import make_video
from nle import nethack
from minihack import RewardManager
import torch

def normalize_glyphs(state):
    glyphs = state["glyphs"]
    glyphs = glyphs/glyphs.max()
    return torch.from_numpy(glyphs.reshape((1,1,21,79))).squeeze(0)
def glyph_pos(glyphs, glyph):
    glyph_positions = np.where(np.asarray(glyphs) == glyph)
    assert len(glyph_positions) == 2
    if glyph_positions[0].shape[0] == 0:
        return None
    return np.array([glyph_positions[0][0], glyph_positions[1][0]], dtype=np.float32)


def distance_to_staircase_reward(env, prev_obs, action, current_obs):
    glyphs = current_obs[env._observation_keys.index("chars")]
    cur_pos = glyph_pos(glyphs, ord("@"))
    staircase_pos = glyph_pos(glyphs, ord(">"))
    if staircase_pos is None:
        # Staircase has been reached
        return 0.0
    distance = np.linalg.norm(cur_pos - staircase_pos)
    distance /= np.max(glyphs.shape)
    return -distance


def go_right_bonus(env, prev, action, curr):
    # Get the x coord of the @
    glyphs = curr[env._observation_keys.index("chars")]
    cur_pos = glyph_pos(glyphs, ord("@"))
    prev_pos = glyph_pos(glyphs, ord("@"))

    if cur_pos is None or prev_pos is None:
        return 0

    # Get the x coord of cur_pos
    cur_x = cur_pos[1]
    prev_x = prev_pos[1]

    # return the reward for moving to the right
    return 0.001 * (cur_x - prev_x)

def same_message(env, prev, action, curr):
    if prev[env._observation_keys.index("message")].all() == \
        curr[env._observation_keys.index("message")].all() and prev[env._observation_keys.index("message")].all() != 0:
        return -0.3
    return 0.0

def  check_float(env, prev, action, curr):
    # if "float" in curr[env._observation_keys.index("message")] or "Float" in
    st = ""
    for n in curr[env._observation_keys.index("message")]:
        st += chr(n)

    if "float" in st or "Float" in st:
        return 1.0

    return 0.0

hyper_params = {
    "seed": 69,  # which seed to use
    "env": "MiniHack-LavaCross-Levitate-Potion-Inv-Full-v0",  # name of the game
    "replay-buffer-size": int(1e6),  # replay buffer size
    "learning-rate": 0.001,  # learning rate for Adam optimizer
    "discount-factor": 0.99,  # discount factor
    "num-steps": int(30000),  # total number of steps to run the environment for
    "batch-size": 256,  # number of transitions to optimize at the same time
    "learning-starts": 1000,  # number of steps before learning starts
    "learning-freq": 2,  # number of iterations between every optimization step
    "use-double-dqn": True,  # use double deep Q-learning
    "target-update-freq": 1000,  # number of iterations between every target network update
    "eps-start": 1.0,  # e-greedy start threshold
    "eps-end": 0.4,  # e-greedy end threshold
    "eps-fraction": 0.95,  # fraction of num-steps
    "print-freq": 10,
}


np.random.seed(hyper_params["seed"])
random.seed(hyper_params["seed"])

# ACTIONS = nethack.CompassDirection.all_directions() + nethack.Command.all_directions()
ACTIONS = (
    nethack.CompassDirection.N,
    nethack.CompassDirection.E,
    nethack.CompassDirection.S,
    nethack.CompassDirection.W,
    nethack.Command.QUAFF,
    nethack.Command.FIRE,
    # nethack.Command.PICKUP,
    # nethack.Command.APPLY
    )

reward_manager = RewardManager()

reward_manager.add_eat_event("apple", reward = 1.0)
reward_manager.add_message_event(["direction", "Direction", "wall.", "stone"], reward = -1.0, terminal_required=False)
reward_manager.add_message_event(["float", "Float", ""], reward = 1.0, terminal_required=False)
# reward_manager.add_message_event(["What do you want to drink? [f or ?*]"], reward = 1.0, terminal_required=False)
# reward_manager.add_message_event(["You start to float in the air!"], reward = 2.0, terminal_required=False)
reward_manager.add_custom_reward_fn(go_right_bonus)
reward_manager.add_custom_reward_fn(distance_to_staircase_reward)
# reward_manager.add_custom_reward_fn(check_float)
# reward_manager.add_custom_reward_fn(same_message)

env = gym.make(hyper_params["env"],
                observation_keys = ['pixel', 'message', 'glyphs'],
                # penalty_time=-0.1,
                # penalty_step=-0.1,
                reward_lose=-3,
                reward_win=5,
                # seeds = hyper_params["seed"],
                actions = ACTIONS,
                reward_manager=reward_manager
                )

replay_buffer = ReplayBuffer(hyper_params["replay-buffer-size"])

agent = DQNAgent(
    env.observation_space["glyphs"],
    env.action_space,
    replay_buffer,
    hyper_params["use-double-dqn"],
    hyper_params["learning-rate"],
    hyper_params["batch-size"],
    hyper_params["discount-factor"]
)

eps_timesteps = hyper_params["eps-fraction"] * float(hyper_params["num-steps"])
episode_rewards = [0.0]

state = env.reset() #must be pixels only



#Mike code
lava_count = 0
key_count = 0
last_action = None
prev_action = None

old_best=-10000
new_best=-10000

best_model = None

# You move over some lava.

for t in range(hyper_params["num-steps"]):
    fraction = min(1.0, float(t) / eps_timesteps)
    eps_threshold = hyper_params["eps-start"] + fraction * (
        hyper_params["eps-end"] - hyper_params["eps-start"]
    )

    sample = random.random()
    if sample <= eps_threshold:
        action = env.action_space.sample()
    else:
        action = agent.act(normalize_glyphs(state))

    # st  = ""
    # for n in state["message"]:
    #     st += chr(n)

    # if "What do you want to drink? [f or ?*]" in st:
    #     # print("Set action to 5")
    #     action = 5

    # prev_action = last_action
    # last_action = action

    # if prev_action == 4:
    #     action = 5

    next_state, reward, done, _ = env.step(action)

    # st  = ""
    # for n in next_state["message"]:
    #     st += chr(n)
    # print(st)

    # next_state = normalize_glyphs(next_state)
    agent.replay_buffer.add(normalize_glyphs(state), action, reward, normalize_glyphs(next_state), float(done))
    state = next_state

    episode_rewards[-1] += reward
    if done:
        state = env.reset()

        new_best=episode_rewards[-1]
        episode_rewards.append(0.0)
        lava_count = 0
        key_count = 0
        last_action = None
        prev_action = None

    if (
        t > hyper_params["learning-starts"]
        and t % hyper_params["learning-freq"] == 0
    ):
        agent.optimise_td_loss()

    if (
        t > hyper_params["learning-starts"]
        and t % hyper_params["target-update-freq"] == 0
    ):
        agent.update_target_network()

    num_episodes = len(episode_rewards)

    if(
        new_best>old_best
    ):
        new_best=old_best
        # make_video(env, agent, 30, 30, hyper_params["env"], 1000, f"video_{t}_{new_best}.mp4")
        best_model = agent
        torch.save(agent, 'bestOne')


    if (
        done
        and hyper_params["print-freq"] is not None
        and len(episode_rewards) % hyper_params["print-freq"] == 0
    ):
        mean_100ep_reward = round(np.mean(episode_rewards[-101:-1]), 1)
        print("********************************************************")
        print("steps: {}".format(t))
        print("episodes: {}".format(num_episodes))
        print("mean 100 episode reward: {}".format(mean_100ep_reward))
        print("% time spent exploring: {}".format(int(100 * eps_threshold)))
        print("********************************************************")


# # torch.save(agent, 'best2')
# #Load the model in
# agent = torch.load('best2')
for r in range(3):
    make_video(env, best_model, 30, 30, hyper_params["env"], 1000, f"video_{r}_best.mp4")
    make_video(env, agent, 30, 30, hyper_params["env"], 1000, f"video_{r}_agent.mp4")

KeyboardInterrupt: ignored